# Time Series Forecasting using facebook prophet

In [0]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation



INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

# Data Pre-processing

In [0]:
df2['Warehouse']=df2['Warehouse'].astype(str)
df3['Warehouse']=df3['Warehouse'].astype(str)
df1['Date']=pd.to_datetime(df1['Date'])
df1['Week']=pd.to_datetime(df1['Week'])
df5=pd.merge(pd.merge(df2,df3,on=['FiscalDayOfWeek','Warehouse'], how='left'),df1,on=['FiscalDayOfWeek'])
df5=df5[['Date','Week','FiscalDayOfWeek','Warehouse','Percentage','Fedex%','UPS%','Other%']]
df5 = df5.rename(columns = {'Week': 'ds', 'Warehouse': 'warehouse'})
df5['Percentage']=df5['Percentage'].astype('float')
df5['Fedex%']=df5['Fedex%'].astype('float')
df5['UPS%']=df5['UPS%'].astype('float')
df5['Other%']=df5['Other%'].astype('float')
df['Qty']=df['Qty'].astype('int')
df['Week']=pd.to_datetime(df['Week'])
df_units=df[['Week','FiscalWeek','Warehouse','Qty']]
df_units.sort_values(['Week'],inplace=True)
df_units=df_units.groupby(['Week','FiscalWeek','Warehouse'])['Qty'].sum().unstack().reset_index()
df_units.fillna(0,inplace=True)


/local_disk0/tmp/1603217241117-0/PythonShell.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 import traceback
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

# Building function to adjust covid period
# Building function to run the profit model, predict and evaluate the model performance

In [0]:

# Functions developed to feature engineering, model building, prediction and cross validation 
def prep(whs,start_date,end_date,start_week,end_week,y,x,rate,beginning):
    table1=df_units[['Week','FiscalWeek',whs]][(df_units['Week']>=start_date) & (df_units['Week']<=end_date)]
    table2=df_units[['Week','FiscalWeek',whs]][(df_units['FiscalWeek']>=start_week) & (df_units['FiscalWeek']<=end_week) & (df_units['Week'].dt.year==2019)]
    table3=pd.merge(table2[['FiscalWeek',whs]],table1[['Week','FiscalWeek',whs]],how='inner',on='FiscalWeek')
    table3['Rate']=((table3[y]-table3[x])/table3[x])
    
    table4=df_units[['Week','FiscalWeek',whs]]
    table4=pd.merge(table4,table3[['Week','Rate']],how='left',on='Week')
    table4.fillna(0,inplace=True)
    table4['new units']=np.where(table4['Rate']==0,table4[whs],(table4[whs]/(1+table4['Rate']))*rate)
    table5=table4[['Week','new units']][table4['Week']>='10/20/2018']
    table5.columns=['ds','y']
    return table5

  

def model(data,changepoints,fourier1,fourier2,prior):
    m=Prophet(growth="linear",yearly_seasonality=False,n_changepoints=changepoints).add_seasonality(name='querterly',period=365.25/4,fourier_order=fourier1,prior_scale=prior).add_seasonality(name='yearly',period=365.25,fourier_order=fourier2)
    return m.fit(data)
    
def prediction(model,future_table,forecast_table):
    future_table=model.make_future_dataframe(freq='w',periods=30)
    forecast_table=model.predict(future_table)
    return forecast_table
    
def MAPE(model):  
    cv_result=cross_validation(model,initial=pd.to_timedelta(55,unit="W"),horizon=pd.to_timedelta(20,unit="W"))
    MAPE=np.mean(np.abs((cv_result['yhat'].values-cv_result['y'].values)/cv_result['y'].values))*100
    return MAPE
    
#MAPE(model(prep('1','03/28/2020','05/30/2020',13,22,'1_y','1_x','01/01/2017')))
#prediction(model(prep('1','03/28/2020','05/30/2020',13,22,'1_y','1_x','01/01/2017')),'future_rkd','forecast_rkd')


INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0

# Final Prediction

In [0]:
import datetime as dt 
def pred_table(table,pred,whs,date):
  table=pred
  table['warehouse']=whs
  table=table[table.ds>date][['ds','yhat','warehouse']]
  return table



df_final=pd.concat([
pred_table('df_rkd',prediction(model(prep('1','03/28/2020','7/11/2020',13,28,'1_y','1_x',1.5,'1/1/2017'),2,5,15,15),'future_rkd','forecast_rkd'),'1','09/05/2020'),
pred_table('df_lpt',prediction(model(prep('15','03/28/2020','07/11/2020',13,28,'15_y','15_x',1.5,'1/1/2018'),2,5,10,10),'future_lpt','forecast_lpt'),'15','09/05/2020'),
pred_table('df_adv',prediction(model(prep('17','03/28/2020','07/11/2020',13,28,'17_y','17_x',1.4,'01/01/2018'),2,5,15,15),'future_adv','forecast_adv'),'17','09/05/2020'),
pred_table('df_ecr',prediction(model(prep('ECR','03/28/2020','07/11/2020',13,28,'ECR_y','ECR_x',1.45,'1/1/2018'),2,5,15,15),'future_ecr','forecast_ecr'),'ECR','09/05/2020'),
pred_table('df_red',prediction(model(prep('5','03/28/2020','07/11/2020',13,28,'5_y','5_x',1.50,'1/1/2018'),1,5,10,10),'future_red','forecast_red'),'5','09/05/2020'),
pred_table('df_msq',prediction(model(prep('28','03/28/2020','07/11/2020',13,28,'28_y','28_x',1.70,'1/1/2018'),2,5,17,17),'future_msq','forecast_msq'),'28','09/05/2020'),
pred_table('df_tac',prediction(model(prep('42','03/28/2020','07/11/2020',13,28,'42_y','42_x',1.0,'1/1/2018'),1,5,10,10),'future_tac','forecast_tac'),'42','09/05/2020')]
          )

df_prediction=pd.merge(df5,df_final,on=['ds','warehouse'],how='inner')
#df_prediction['Pct']=df_prediction['Pct'].astype('float')
df_prediction['FEDEX_unit']=df_prediction['Percentage']*df_prediction['yhat']*df_prediction['Fedex%']
df_prediction['UPS_unit']=df_prediction['Percentage']*df_prediction['yhat']*df_prediction['UPS%']
df_prediction['Other_unit']=df_prediction['Percentage']*df_prediction['yhat']*df_prediction['Other%']
df_prediction['Date_Run'] = pd.to_datetime('today')
df_prediction.sort_values(['ds','Date'],inplace=True)
df_prediction.columns=['Date','Week','Day','Warehouse','Percentage','FEDEX%','UPS%','Other%','Prediction','FEDEX_unit','UPS_unit','Other_unit','Date_run']
df=spark.createDataFrame(df_prediction)



INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
WARNING:fbprophet.models:Optimization terminated abnormally. Falling back to Newton.
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
WARNING:fbprophet.models:Optimization terminated abnormally. Falling back to Newton.
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id p0
INFO:py4j.java_gateway:Received command c on object id

In [0]:
df.show()

+-------------------+-------------------+---+---------+----------+--------+--------+------+------------------+------------------+------------------+----------+--------------------+
 Date| Week|Day|Warehouse|Percentage| FEDEX%| UPS%|Other%| Prediction| FEDEX_unit| UPS_unit|Other_unit| Date_run|
+-------------------+-------------------+---+---------+----------+--------+--------+------+------------------+------------------+------------------+----------+--------------------+
2020-10-18 00:00:00|2020-10-18 00:00:00| 1| 28| 0.069952|0.294121|0.705878| 0.0| 8122.563212318431|167.11647623202228|401.07249740653475| 0.0|2020-10-20 20:14:...|
2020-10-18 00:00:00|2020-10-18 00:00:00| 1| ECR| 0.035095|0.388022|0.611977| 0.0| 8667.991087338429|118.03751358677377|186.16532942022116| 0.0|2020-10-20 20:14:...|
2020-10-18 00:00:00|2020-10-18 00:00:00| 1| 17| 0.118093|0.326505|0.673494| 0.0|10846.289681067065|418.21074905992805| 862.6588573754375| 0.0|2020-10-20 20:14:...|
2020-10-18 00:00:00|2020-10-18 00:00:00| 1| 15| 0.11215|0.242567|0.757432| 0.0|11587.807241808365| 315.2334225389413| 984.3378600572847| 0.0|2020-10-20 20:14:...|
2020-10-18 00:00:00|2020-10-18 00:00:00| 1| 1| 0.074137|0.704605|0.295394| 0.0| 9751.108679925974| 509.3715980756282|213.54562321009942| 0.0|2020-10-20 20:14:...|
2020-10-18 00:00:00|2020-10-18 00:00:00| 1| 5| 0.076639|0.257425|0.742574| 0.0| 12854.03680639056|253.59465161276844| 731.5248900716709| 0.0|2020-10-20 20:14:...|
2020-10-19 00:00:00|2020-10-18 00:00:00| 2| 28| 0.198029|0.189541|0.810458| 0.0| 8122.563212318431| 304.8772804614184| 1303.624181407718| 0.0|2020-10-20 20:14:...|
2020-10-19 00:00:00|2020-10-18 00:00:00| 2| ECR| 0.277037|0.285435|0.714564| 0.0| 8667.991087338429| 685.4305494533336|1715.9212960553957| 0.0|2020-10-20 20:14:...|
2020-10-19 00:00:00|2020-10-18 00:00:00| 2| 17| 0.177477| 0.28925|0.710749| 0.0|10846.289681067065| 556.7966913654594|1368.1683373943263| 0.0|2020-10-20 20:14:...|
2020-10-19 00:00:00|2020-10-18 00:00:00| 2| 15| 0.183531|0.281044|0.718955| 0.0|11587.807241808365| 597.7024158633085|1529.0173083111717| 0.0|2020-10-20 20:14:...|
2020-10-19 00:00:00|2020-10-18 00:00:00| 2| 1| 0.255127|0.294361|0.705638| 0.0| 9751.108679925974| 732.3027899985515| 1755.465826413818| 0.0|2020-10-20 20:14:...|
2020-10-19 00:00:00|2020-10-18 00:00:00| 2| 5| 0.194959|0.262597|0.737402| 0.0| 12854.03680639056| 658.0707504416766|1847.9369052852592| 0.0|2020-10-20 20:14:...|
2020-10-19 00:00:00|2020-10-18 00:00:00| 2| 42| 0.181901|0.266625|0.733374| 0.0| 525.4660855508952| 25.48476576381041| 70.09794508117655| 0.0|2020-10-20 20:14:...|
2020-10-20 00:00:00|2020-10-18 00:00:00| 3| 28| 0.201876|0.481819| 0.51818| 0.0| 8122.563212318431| 790.0629803927378| 849.6859509066867| 0.0|2020-10-20 20:14:...|
2020-10-20 00:00:00|2020-10-18 00:00:00| 3| ECR| 0.185432|0.436378|0.563621| 0.0| 8667.991087338429| 701.4003626270102| 905.9209533574061| 0.0|2020-10-20 20:14:...|
2020-10-20 00:00:00|2020-10-18 00:00:00| 3| 17| 0.164533| 0.30488|0.695119| 0.0|10846.289681067065| 544.0804882193659|1240.4903073030616| 0.0|2020-10-20 20:14:...|
2020-10-20 00:00:00|2020-10-18 00:00:00| 3| 15| 0.163592|0.257354|0.742645| 0.0|11587.807241808365| 487.8589165986468|1407.8117500307048| 0.0|2020-10-20 20:14:...|
2020-10-20 00:00:00|2020-10-18 00:00:00| 3| 1| 0.181137|0.406466|0.593533| 0.0| 9751.108679925974| 717.9354381630324|1048.3493685061458| 0.0|2020-10-20 20:14:...|
2020-10-20 00:00:00|2020-10-18 00:00:00| 3| 5| 0.17066| 0.40712|0.592862| 0.0| 12854.03680639056| 893.086898391661|1300.5435369283673| 0.0|2020-10-20 20:14:...|
2020-10-20 00:00:00|2020-10-18 00:00:00| 3| 42| 0.260359|0.273245|0.726754| 0.0| 525.4660855508952| 37.38260051406828| 99.42708724405269| 0.0|2020-10-20 20:14:...|
+-------------------+-------------------+---+---------+----------+--------+--------+------+------------------+------------------+------------------+----------+--------------------+
only showing top 20 rows

INFO:py4j.java_gateway:Received command c o